In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os, shutil, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
from tqdm import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader, Subset, sampler, random_split
from torchvision import datasets, transforms, models

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Cuda available:", torch.cuda.is_available())

seed = 1
np.random.seed(seed)
torch.manual_seed(seed)
if device.type == 'cuda':
    torch.cuda.manual_seed_all(seed)

In [ ]:
model_settings = {
    "resnet18": {'target_layer': 'layer4.1.conv2'}, 
    "resnet50": {'target_layer': 'layer4.2.conv3'}, 
    "vgg16": {'target_layer': 'features.conv5_3'},   # 'features.28'
    "alexnet": {'target_layer': 'conv5'}
}

model_dataset_settings = {
    "resnet18_indoor_bedroom_kitchen": {'load_model': False, 'num_classes': 2, 'base_num_classes': 1000},
    "resnet18_places_bedroom_kitchen": {'load_model': True, 'num_classes': 2, 'base_num_classes': 365},
    "resnet18_places_bedroom_kitchen_livingroom": {'load_model': True, 'num_classes': 3, 'base_num_classes': 365},
    "resnet18_places_coffeeshop_restaurant": {'load_model': True, 'num_classes': 2, 'base_num_classes': 365},
    "resnet18_imagenet_minivan_pickup": {'load_model': False, 'num_classes': 2, 'base_num_classes': 1000, 'excluded_concepts': ['pickup', 'van']},   # ['car', 'bus', 'coach', 'truck', 'van']
    "resnet18_imagenet_laptop_mobile": {'load_model': False, 'num_classes': 2, 'base_num_classes': 1000, 'excluded_concepts': ['laptop', 'mobile', 'computer']},
    "resnet50_places_bedroom_kitchen": {'load_model': True, 'num_classes': 2, 'base_num_classes': 365}, 
    "resnet50_places_bedroom_kitchen_livingroom": {'load_model': True, 'num_classes': 3, 'base_num_classes': 365},
    "resnet50_places_coffeeshop_restaurant": {'load_model': True, 'num_classes': 2, 'base_num_classes': 365},
    "resnet50_imagenet_minivan_pickup": {'load_model': False, 'num_classes': 2, 'base_num_classes': 1000, 'excluded_concepts': ['pickup', 'van']},
    "vgg16_places_bedroom_kitchen": {'load_model': True, 'num_classes': 2, 'base_num_classes': 365}, 
    "vgg16_places_coffeeshop_restaurant": {'load_model': True, 'num_classes': 2, 'base_num_classes': 365},
    "vgg16_places_bedroom_kitchen_livingroom": {'load_model': True, 'num_classes': 3, 'base_num_classes': 365},
    "vgg16_imagenet_minivan_pickup": {'load_model': False, 'num_classes': 2, 'base_num_classes': 1000, 'excluded_concepts': ['pickup', 'van']}
}

settings = {
    "model_settings": model_settings,
    "model_dataset_settings": model_dataset_settings
}

settings_path = '/content/drive/My Drive/Python Projects/POEM Pipeline Results/settings.pkl' 
with open(settings_path, 'wb') as f:
    pickle.dump(settings, f)

In [ ]:
current_setting_path = '/content/drive/My Drive/Python Projects/POEM Pipeline Results/current_setting.txt'
with open(current_setting_path, 'r') as f:
    current_setting_title = f.read().splitlines()[0]
    print('Current setting:', current_setting_title)

title_parts = current_setting_title.split('_')
model_name = title_parts[0]   # custom, resnet18, resnet50, vgg16, alexnet
dataset_name = '_'.join(title_parts[1:])   # cifar10, imagenette, imagewoof, indoor, places, imagenet
dataset_pure_name = dataset_name.split('_')[0]

current_setting = model_dataset_settings[current_setting_title] 
load_model_from_disk = current_setting['load_model']   # Is true in case of having a model file pretrained on the target dataset (e.g. Places365) rather than the default dataset of Torchvision which is ImageNet
num_classes = current_setting['num_classes']
base_num_classes = current_setting['base_num_classes']

pretrain_mode = 'feature_extraction'   # full_fine_tuning, partial_fine_tuning, feature_extraction
target_classes = []   # ['n02086240', 'n02087394']

train_ratio = 0.7
image_size = 224
batch_size = 64
epochs = 100
stop_patience = 10
cnn_dropout = 0.0
dense_dropout = 0.0

norm_mean = (0.485, 0.456, 0.406)
norm_std = (0.229, 0.224, 0.225)

In [ ]:
def save_imported_packages(packages_path):

    # Saving imported packages and their versions: 
    import sys
    modules_info = []

    for module in sys.modules:
        if len(module.split('.')) > 1:   # ignoring subpackages
            continue

        try:
            modules_info.append((module, sys.modules[module].__version__))
        except:
            try:
                if type(sys.modules[module].version) is str:
                    modules_info.append((module, sys.modules[module].version))
                else:
                    modules_info.append((module, sys.modules[module].version()))
            except:
                try:
                    modules_info.append((module, sys.modules[module].VERSION))
                except:
                    pass

    modules_info.sort(key=lambda x: x[0])
    with open(packages_path, 'w') as f:
        for m in modules_info:
            f.write('{} {}\n'.format(m[0], m[1]))

In [ ]:
class ImageDataset (Dataset):
    
    def __init__(self, images_path, file_names, labels, transform):
        self.images_path = images_path
        self.file_names = file_names
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, index):
        fname = self.file_names[index]
        path = os.path.join(self.images_path, fname)
        img = Image.open(path)
        
        if self.transform:
            img = self.transform(img)
        
        if self.labels is not None:
            label = self.labels[index]
            return img, label
        
        return img, fname

In [ ]:
# This class helps to be able to access class index, targets and file paths of the dataset, which are absent in the Subset instance itself
class ImageSubset (Subset):

    def __init__(self, dataset, indexes):
        super().__init__(dataset, indexes)
        self.class_to_idx = dataset.class_to_idx
        self.targets = dataset.targets
        self.samples = [s for i,s in enumerate(dataset.samples) if i in indexes]

In [ ]:
def prepare_data (images_path=None, labels_path=None): 

    train_transform = transforms.Compose([
        #transforms.RandomResizedCrop(image_size),
        transforms.Resize(256),
        transforms.RandomCrop(image_size),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=norm_mean, std=norm_std)
    ])
    
    valid_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=norm_mean, std=norm_std)
    ])

    train_set = None
    valid_set = None
    train_dataset_dir = dataset_name
    valid_dataset_dir = dataset_name

    if dataset_pure_name in ['places']:
        train_dataset_dir += '/train'
        valid_dataset_dir += '/train'   # Because the validation set is very small, we just use and split the train set
    elif dataset_pure_name in ['imagenette', 'imagewoof']:
        train_dataset_dir += '/train'
        valid_dataset_dir += '/val'
    elif dataset_pure_name in ['cifar10']:
        train_dataset_dir = './data'
        valid_dataset_dir = './data'

    # Loading or computing the train and validation sets for different datasets: 
    if dataset_pure_name in ['indoor', 'places', 'imagenet']:
        train_set = datasets.ImageFolder(root=train_dataset_dir, transform=train_transform)
        valid_set = datasets.ImageFolder(root=valid_dataset_dir, transform=valid_transform)

        data_size = len(train_set)
        indexes = list(range(data_size))
        np.random.shuffle(indexes)
        train_size = int(train_ratio * data_size)
        train_indexes = indexes[:train_size]
        valid_indexes = indexes[train_size:]

        train_set = ImageSubset(train_set, train_indexes)
        valid_set = ImageSubset(valid_set, valid_indexes)

        # Can't use the code below, because training and validation data have different transforms to apply: 
        # valid_size = data_size - train_size
        # train_set, valid_set = random_split(train_set, [train_size, valid_size], generator=torch.Generator().manual_seed(seed))

    elif dataset_pure_name in ['imagenette', 'imagewoof']:
        train_set = datasets.ImageFolder(root=train_dataset_dir, transform=train_transform)
        valid_set = datasets.ImageFolder(root=valid_dataset_dir, transform=valid_transform)

    elif dataset_pure_name == 'cifar10':
        train_set = datasets.CIFAR10(root=train_dataset_dir, train=True, transform=train_transform, download=True)
        valid_set = datasets.CIFAR10(root=valid_dataset_dir, train=False, transform=valid_transform, download=True)

    else: 
        data = pd.read_csv(labels_path)
        file_names = data['File'].tolist()
        labels = data['Label'].tolist()

        train_file_names, valid_file_names, train_labels, valid_labels = train_test_split(file_names, labels, test_size= 1.0 - train_ratio, 
                                                                                          random_state=seed, stratify=labels)
        
        train_set = ImageDataset(images_path, train_file_names, train_labels, transform=train_transform)
        valid_set = ImageDataset(images_path, valid_file_names, valid_labels, transform=valid_transform)

    # In case we want to focus on selected target classes among all the classes in the dataset: 
    if (target_classes != None) and (len(target_classes) > 0):
        class_indexes_dict = train_set.class_to_idx
        train_indexes = train_set.targets
        valid_indexes = valid_set.targets

        target_class_indexes = [v for k,v in class_indexes_dict.items() if k in target_classes]
        target_train_indexes = [i for i,x in enumerate(train_indexes) if x in target_class_indexes]
        target_valid_indexes = [i for i,x in enumerate(valid_indexes) if x in target_class_indexes]
        print('target_class_indexes:', target_class_indexes)
        print('target_train_indexes: {}, target_valid_indexes: {}'.format(len(target_train_indexes), len(target_valid_indexes)))

        train_set = ImageSubset(train_set, target_train_indexes)
        valid_set = ImageSubset(valid_set, target_valid_indexes)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)
    
    return train_loader, valid_loader

In [ ]:
def save_data_subset (data_subset, dataset_dir, target_dataset_dir):

    images_info = data_subset.samples
    class_to_idx = data_subset.class_to_idx
    print('class_to_idx:', class_to_idx)
    #idx_to_class = {v:k for k,v in class_to_idx.items()}
    class_names = list(class_to_idx.keys())
    idx_to_class_dir = {}

    if not os.path.exists(target_dataset_dir):
        print('Making target dataset dir:', target_dataset_dir)
        os.makedirs(target_dataset_dir)

    for cname in class_names:
        idx = class_to_idx[cname]
        target_class_dir = target_dataset_dir + '/' + cname
        idx_to_class_dir[idx] = target_class_dir
        if not os.path.exists(target_class_dir):
            os.makedirs(target_class_dir)

    print('idx_to_class_dir:', idx_to_class_dir)

    for i,(path, label) in enumerate(images_info):
        target_class_dir = idx_to_class_dir[label]
        ind = path.rfind('/')
        fname = path[ind+1:]
        target_path = target_class_dir + '/' + fname
        if i == 0:
            print('Label: {}, fname: {}, path: {}, target_path: {}'.format(label, fname, path, target_path))
        shutil.copy(path, target_path)

In [ ]:
class ConvNet (nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )

        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=cnn_dropout)
        )

        self.conv_layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.conv_layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=cnn_dropout)
        )

        self.conv_layer5 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )

        self.conv_layer6 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p=cnn_dropout)
        )

        self.fc_layer1 = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dense_dropout)
        )

        self.classifier = nn.Linear(256, num_classes)
        
        
    def forward(self, x): 
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.conv_layer5(out)
        out = self.conv_layer6(out)

        out = out.view(-1, 128 * 4 * 4)
        out = self.fc_layer1(out)
        out = self.classifier(out)
        
        return out
        

In [ ]:
def vgg16_model (*args, **kwargs):

    # A version of vgg16 model where layers are given their research names: 
    model = models.vgg16(*args, **kwargs)
    model.features = nn.Sequential(OrderedDict(zip([
        'conv1_1', 'relu1_1',
        'conv1_2', 'relu1_2',
        'pool1',
        'conv2_1', 'relu2_1',
        'conv2_2', 'relu2_2',
        'pool2',
        'conv3_1', 'relu3_1',
        'conv3_2', 'relu3_2',
        'conv3_3', 'relu3_3',
        'pool3',
        'conv4_1', 'relu4_1',
        'conv4_2', 'relu4_2',
        'conv4_3', 'relu4_3',
        'pool4',
        'conv5_1', 'relu5_1',
        'conv5_2', 'relu5_2',
        'conv5_3', 'relu5_3',
        'pool5'],
        model.features)))

    model.classifier = nn.Sequential(OrderedDict(zip([
        'fc6', 'relu6',
        'drop6',
        'fc7', 'relu7',
        'drop7',
        'fc8a'],
        model.classifier)))

    return model

In [ ]:
def pretrained_model (base_model_file=None):
    
    # When loading the model from disk, the "pretrained" parameter should be false and "num_classes" should be set based on the classes in the loaded model file
    # When not loading the model from disk, the "pretrained" parameter should be true, and "num_classes" should not be provided, 
    # or it should be equal to the classes in the pretrained model (e.g. ImageNet in PyTorch)
    pretrained = (not load_model_from_disk)
    model = None
    partial_target_layers = []

    if model_name == 'vgg16':
        model = vgg16_model(pretrained=pretrained, num_classes=base_num_classes)
    else:
        model = models.__dict__[model_name](pretrained=pretrained, num_classes=base_num_classes)

    if load_model_from_disk:
        #model = models.__dict__[model_name](num_classes=base_num_classes)
        checkpoint = torch.load(base_model_file)
        statedict = checkpoint
        if 'state_dict' in checkpoint:
            statedict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
        model.load_state_dict(statedict)
    # else:
    #     model = models.__dict__[model_name](pretrained=pretrained)

    if model_name == 'resnet18':
        partial_target_layers = ['layer3', 'layer4', 'avgpool']

    if pretrain_mode == 'partial_fine_tuning':
        for name, child in model.named_children():
            if name in partial_target_layers:
                for p in child.parameters():
                    p.requires_grad = True
            else:
                for p in child.parameters():
                    p.requires_grad = False
    elif pretrain_mode == 'feature_extraction':
        for p in model.parameters():
            p.requires_grad = False

    if model_name.startswith('resnet'):
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == 'alexnet':
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif model_name == 'vgg16':
        model.classifier.fc8a = nn.Linear(model.classifier.fc8a.in_features, num_classes)   # model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
        
    return model

In [ ]:
def train_epoch (model, optimizer, criterion, train_loader, valid_loader):
    
    model.train()
    train_loss = 0
    train_acc = 0
    train_steps = 0
    train_size = 0
    
    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images = images.to(device)
        labels = labels.to(device)
    
        optimizer.zero_grad()
        
        #labels = labels.unsqueeze(1).float()   # reshape to 2 dimensions
        output = model(images)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()
        
        loss = loss.cpu()
        train_loss += loss.item()
        _, preds = torch.max(output, 1)
        preds = preds.cpu()
        labels = labels.cpu()
        train_acc += (preds == labels).float().sum()

        train_steps += 1
        train_size += len(preds)
    
    train_loss = train_loss / train_steps
    train_acc = train_acc / train_size   #len(train_loader.dataset)
    
    model.eval()
    val_loss = 0
    val_acc = 0
    val_steps = 0
    val_size = 0
    
    with torch.no_grad():
        for i, (images, labels) in tqdm(enumerate(valid_loader)):
            images = images.to(device)
            labels = labels.to(device)
            
            #labels = labels.unsqueeze(1).float()
            output = model(images)
            vloss = criterion(output, labels)
            
            vloss = vloss.cpu()
            val_loss += vloss.item()
            
            _, preds = torch.max(output, 1)
            preds = preds.cpu()
            labels = labels.cpu()
            val_acc += (preds == labels).float().sum()

            val_steps += 1
            val_size += len(preds)
            
    val_loss = val_loss / val_steps
    val_acc = val_acc / val_size   #len(valid_loader.dataset)
    
    return train_loss, train_acc, val_loss, val_acc

In [ ]:
def train (model, model_path, train_loader, valid_loader): 

    model_params = [p for p in model.parameters() if p.requires_grad]
    print('Number of params to learn:', len(model_params))
    
    optimizer = optim.Adam(model_params, lr=0.001, weight_decay=1e-6)
    criterion = nn.CrossEntropyLoss()

    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    best_acc = 0
    best_epoch = 0
    no_progress = 0
    
    for e in range(epochs):
        print('\nEpoch {}/{}'.format(e+1, epochs))
        train_loss, train_acc, val_loss, val_acc = train_epoch(model, optimizer, criterion, train_loader, valid_loader)
        
        print('loss: {:.3f} - acc: {:.3f} - val loss: {:.3f} - val acc: {:.3f}'.format(train_loss, train_acc, val_loss, val_acc))
        
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_epoch = e
            no_progress = 0
            torch.save(model.state_dict(), model_path)
        else:
            no_progress += 1
            
        if no_progress >= stop_patience:
            print('Finished training in epoch {} because of no progress in {} consecutive epochs'.format(e, no_progress))
            break;
    
    print('Best validation accuracy: {:.3f} (epoch {})'.format(best_acc, best_epoch))
    return train_losses, train_accs, val_losses, val_accs
        

In [ ]:
def plot_results (train_losses, train_accs, val_losses, val_accs):
    
    # loss: 
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.title('Loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train loss', 'validation loss'], loc='upper right')
    plt.show()
    
    # accuracy: 
    plt.plot(train_accs)
    plt.plot(val_accs)
    plt.title('Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train accuracy', 'validation accuracy'], loc='lower right')
    plt.show()

In [ ]:
dataset_file = dataset_name + '.zip'
drive_dataset_dir = '/content/drive/My Drive/Python Projects/Datasets/' + dataset_file
!cp "$drive_dataset_dir" '.'
!unzip -qq -n $dataset_file -d '.'

base_model_file = None
if load_model_from_disk:
    base_model_file = model_name + '_' + dataset_pure_name + '.pth'
    base_model_path = "/content/drive/My Drive/Python Projects/Pretrained Models/" + base_model_file
    !cp "$base_model_path" '.'

# if dataset_name == 'imagenette':
#     !cp '/content/drive/My Drive/Python Projects/Other Data/imagenette2-320.tgz' '.'
#     !tar -xf 'imagenette2-320.tgz'

# elif dataset_name == 'imagewoof':
#     !cp '/content/drive/My Drive/Python Projects/Other Data/imagewoof2-320.tgz' '.'
#     !tar -xf 'imagewoof2-320.tgz'

# elif dataset_name == 'indoor':
#     !cp '/content/drive/My Drive/Python Projects/Other Data/indoor_subset.zip' '.'
#     !unzip 'indoor_subset.zip' -d '.'

# elif dataset_name == 'places2':
#     !cp '/content/drive/My Drive/Python Projects/Other Data/places2.zip' '.'
#     !unzip 'places2.zip' -d '.'

In [ ]:
train_loader, valid_loader = prepare_data()
valid_dataset_dir = 'dataset'
save_data_subset(valid_loader.dataset, dataset_name, valid_dataset_dir)

model = None
if model_name == 'custom':
    model = ConvNet()
else:
    model = pretrained_model(base_model_file)

print(model)
model = model.to(device)

model_path = 'model.pth'   # model_name + '_' + dataset_name + '.pth'
train_losses, train_accs, val_losses, val_accs = train(model, model_path, train_loader, valid_loader)

plot_results(train_losses, train_accs, val_losses, val_accs)

In [ ]:
drive_result_path = '/content/drive/My Drive/Python Projects/POEM Pipeline Results/' + model_name + '_' + dataset_name
if not os.path.exists(drive_result_path):
    os.makedirs(drive_result_path)

!cp $model_path '$drive_result_path'

valid_dataset_file = valid_dataset_dir + '.zip'
!zip -qq -r $valid_dataset_file $valid_dataset_dir
!cp $valid_dataset_file '$drive_result_path'

packages_path = drive_result_path + '/pretraining_packages.log'
save_imported_packages(packages_path)